In [16]:
from bs4 import BeautifulSoup
import requests as r
from dataclasses import dataclass

In [32]:
'''
    we're making a class just to store some stock data
    the dataclasses import just auto creates the init function and other base functions 
    that might be needed so we don't have to, it's just a lazy convenient way to get things done
''' 

@dataclass
class Stock:
    ticker: str
    exchange: str
    price: float = 0
    currency: str = "USD"
    usd_price: float = 0

    def __post_init__(self):

        # get_price is a function defined below
        price_info = get_price(self.ticker, self.exchange)

        self.price = price_info['price']
        self.currency = price_info['currency']
        self.usd_price = price_info['usd price']

In [51]:
# position shows us how much of which stock we have, one instance per ticker

@dataclass
class Position:
    stock: Stock
    quantity: int = 0

In [52]:
# portfolio is a combination of positions

@dataclass
class Portfolio:
    positions: list[Position]

    def get_total_value(self):
        total = 0

        for t in self.positions:
            total += t.stock.usd_price * t.quantity

        return total


In [53]:
# https://www.google.com/finance/quote/CAD-USD?hl=en

def price_to_usd(original_currency, value):
    url = f"https://www.google.com/finance/quote/{original_currency}-USD?hl=en"
    resp = r.get(url)
    soup = BeautifulSoup(resp.content)

    # now we need to get the conversion rate
    rate_div = soup.find("div", attrs={"data-last-price": True})
    rate = round(float(rate_div['data-last-price']), 4)

    return round(rate * value, 2)

In [65]:
# https://www.google.com/finance/quote/INTC:NASDAQ?hl=en

def get_price(t, e):
    url = f"https://www.google.com/finance/quote/{t}:{e}?hl=en"
    resp = r.get(url)
    soup = BeautifulSoup(resp.content)

    # we need to target the div that has the data-last-price attribute
    price_div = soup.find("div", attrs={"data-last-price": True})
    price = float(price_div['data-last-price'])
    currency = price_div['data-currency-code']
    usd_price = price_to_usd(currency, price) if currency != "USD" else price

    return {
        "ticker": t,
        "exchange": e,
        "price": price,
        "currency": currency,
        "usd price": usd_price
    }

In [73]:
td = Position(Stock("TD", "TSE"), 10)
msft = Position(Stock("MSFT", "NASDAQ"), 3)

In [74]:
port = Portfolio([td, msft])
port.get_total_value()

1527.1799999999998